<div style="width: 50%; clear: both;">
    <div style="float: left; width: 50%;">
       <img src="https://www.uab.cat/servlet/BlobServer?blobkey=id&blobnocache=true&blobwhere=1345733274171&blobheader=application&blobcol=urlimg&blobtable=Imatge", align="left">
    </div>
</div>

<div style="float: right; width: 50%;">
    <p style="margin: 0; padding-top: 22px; text-align:right;">Garantia de la Informació i Seguretat</p>
    <p style="margin: 0; text-align:right;">Grau en Enginyeria Informàtica</p>
    <p style="margin: 0; text-align:right; padding-button: 100px;">Curs 2024-2025</p>
</div>

</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>

## IMPORTANT!

**Cal desar i lliurar la pràctica posant el grup, noms i NIUs dels membres del grup:**

NÚMERO GRUP = 

NIU_ESTUDIANT_1 =

NOM_ESTUDIANT_1 =

NIU_ESTUDIANT_2 =

NOM_ESTUDIANT_2 =

Per resoldre aquesta pràctica, heu de posar les solucions en aquest mateix
*notebook*, a les cel·les de codi que estan en blanc habilitades per això.
Després de cada cel·la de codi podeu posar una cel·la *markdown* a on
podeu posar la justificació de la resposta donada a la solució. També ho podeu
fer afegint comentaris al codi. No és
obligatori, però sí recomanable que justifiqueu les respostes que
doneu, ja que això facilita la correcció en cas d'una resposta incorrecta
(podent valorar el plantejament), i pot ajudar a descartar possibles
sospites sobre còpies.


# Pràctica 3 - Publicació de microdades

---

Disposem de dades sobre l'activitat econòmica de diverses empreses de Taiwan.
Les dades s'utilitzen per predir la fallida d'empreses i és un
subconjunt obtingut del repositori [UCI Machine Learning
Repository](https://archive.ics.uci.edu/ml/datasets/Taiwanese+Bankruptcy+Prediction).

Suposarem que les dades no han estat prèviament anonimitzades i procedirem
a aplicar diversos mètodes de protecció amb l'objectiu de preservar la
privacitat de les empreses que apareixen en l'estudi. Hem limitat el
conjunt de dades als següents atributs:
- `'ROA (C) before interest and depreciation before interest'`
- `'ROA (B) before interest and depreciation after tax'`
- `'Operating Gross Margin'`
- `'Research and development expense rate'`

Tots ells són de tipus numèric (real).

Podem llegir el fitxer CSV, que es troba en `data/company.csv`, com un
`DataFrame` de *pandas* que podem guardar a la variable `df_compay`.


In [2]:
from typing import Dict, List

import numpy as np
import pandas as pd
import seaborn as sns

df_company = pd.read_csv("data/company.csv")
df_company.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6819 entries, 0 to 6818
Data columns (total 4 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0    ROA(C) before interest and depreciation before interest  6819 non-null   float64
 1    ROA(B) before interest and depreciation after tax        6819 non-null   float64
 2    Operating Gross Margin                                   6819 non-null   float64
 3    Research and development expense rate                    6819 non-null   float64
dtypes: float64(4)
memory usage: 213.2 KB


## 1. Anonimització amb soroll

Per poder començar a treballar amb el conjunt de dades farem una primera
anonimització utilitzant soroll additiu de forma molt senzilla.

### **Exercici 1** [10%]

Afegiu soroll additiu a cadascuna de les 4 variables utilitzant una distribució normal
$N(\mu, \sigma^2)$ amb $\mu = 0$ i com variança prendrem $\sigma^2 = p \sigma_o^2$,
on $\sigma_o^2$ és la variança de la variable que volem protegir i $p$ és un paràmetre que
determina la quantitat de soroll.

Desenvolupeu una funció `noise_add_normal(df, p)` que rep com a paràmetre `df` el
DataFrame a anonimitzar i el paràmetre `p`, i retorna una còpia del DataFrame protegit.

Recordeu que:
- cada variable (columna) tindrà una distribució de soroll independent.
- la funció no pot modificar el DataFrame original.

Finalment, genereu un DataFrame `df_noise_a` com la versió protegida de
`df_company`, amb el paràmetre $p = 0.1$.

*Nota*: funcions que us poden ser d'utilitat:
- `numpy.random.Generator.normal`


In [3]:
def noise_add_normal(df: pd.DataFrame, p: float) -> pd.DataFrame:
    # --- IMPLEMENT YOUR CODE HERE ---
    
    ...
    
    # --------------------------------

In [4]:
df_noise_a = ...

### **Exercici 2** [10%]

En aquest cas heu d'implementar soroll multiplicatiu utilitzant també una
distribució normal. Utilitzarem una distribució normal
$N(\mu, \sigma^2)$ on $\mu = 1$ i $\sigma^2 = p \sigma^2_o$ on
$\sigma^2_o$ és la variança de la variable original i $p$ un paràmetre que
determina la quantitat de soroll.

Implementeu una funció `noise_mul_normal(df, p)` que rep com a paràmetre
`df` el DataFrame a anonimitzar i el paràmetre` p`, i retorna una còpia del
DataFrame protegit. De la mateixa manera que en el cas additiu, recordeu que:
- cada variable (columna) tindrà una distribució de soroll independent.
- la funció no pot modificar el DataFrame original.

Finalment, genereu una versió protegida de les dades amb el paràmetre
$p = 0.1$ i la guardeu a la variable `df_noise_m`.

In [5]:
def noise_mul_normal(df: pd.DataFrame, p: float) -> pd.DataFrame:
    # --- IMPLEMENT YOUR CODE HERE ---
    
    ...
    
    # --------------------------------

In [6]:
df_noise_m = ...

## 2. Mesures de pèrdua d'informació

### **Exercici 3** [20%]

En aquest exercici desenvoluparem diverses mesures de pèrdua d'informació.
Per a això utilitzarem la següent notació:
- $X$: conjunt de dades original. El podem interpretar com una matriu de
mida $n \times d$ on $n$ és el nombre de registres (files) i $d$ el
nombre de variables o atributs (columnes).
- $X'$: conjunt de dades protegit, també $n \times d$.
- $R$: matriu de correlació de les dades originals $X$. Aquesta matriu és una
matriu $d \times d$ on l'element $(i, j)$ és la correlació entre la
variable $V_i$ i $V_j$. Com a mesura de correlació prendrem el coeficient de
correlació de Pearson.
- $R'$: matriu de correlació de les dades protegides $X'$.

En cada cas mesurarem la mitjana de l'error absolut MAE (*mean absolute error*)
i la mitjana de l'error relatiu MRE (*mean relative error*). És a dir:
- Sobre $X$, $X'$:
  - *MAE*: $\frac{1}{nd}\sum_{j=1}^d \sum_{i=1}^n \mid x_{ij} - x'_{ij} \mid$
  - *MRE*: $\frac{1}{nd}\sum_{j=1}^d \sum_{i=1}^n \frac{\mid x_{ij} - x'_{ij} \mid}{\mid x_{ij}\mid}$
- Sobre $R$, $R'$:
  - *MAE*: $\frac{1}{d^2}\sum_{j=1}^d \sum_{i=1}^n \mid r_{ij} - r'_{ij} \mid$
  - *MRE*: $\frac{1}{d^2}\sum_{j=1}^d \sum_{i=1}^n \frac{\mid r_{ij} - r'_{ij} \mid}{\mid r_{ij}\mid}$

Es demana que implementeu la funció
`information_loss(df_original, df_protected)` on `df_original` és el
DataFrame amb les dades originals ($X$) i `df_protected` el DataFrame amb les
dades protegides ($X'$). La funció ha de retornar un diccionari de Python
amb els següents camps:
- `'data-mae'`: valor del MAE sobre $X$ i $X'$
- `'data-mre'`: valor del MRE sobre $X$ i $X'$
- `'corr-mae'`: valor del MAE sobre $R$ i $R'$
- `'corr-mre'`: valor del MRE sobre $R$ i $R'$

Por exemple,

```
{
  'data-mae': 52604277.58657338,
  'data-mre': 0.003232,
  'corr-mae': 0.0023844305276822485,
  'corr-mre': 0.0201830985522146
}
```

**Nota**: El càlcul de MRE pot donar lloc a divisions per zero i, per tant, a valors d'error (NaN) o infinits. No cal que feu res al respecte.

Funcions que us poden ser d'utilitat:
- `pandas.DataFrame.corr`

In [7]:
def information_loss(df_original: pd.DataFrame, df_protected: pd.DataFrame) -> Dict[str, float]:
    # --- IMPLEMENT YOUR CODE HERE ---
    
    ...
    
    # --------------------------------    

## **Exercici 4** [5%]

Ara podem calcular la pèrdua d'informació obtinguda a l'aplicar el soroll
additiu i multiplicatiu anterior (recordem que l'hem aplicat en els dos
casos amb el paràmetre $p = 0.1$).

- Mostra els valors de pèrdua d'informació en cada cas per `df_noise_a`
i `df_noise_m`.
- Mostra de forma gràfica la pèrdua d'informació a l'aplicar soroll additiu
a `df_company` amb valors de $p$ entre $0.1$ i $1.0$ amb un decimal. És a dir, per
$p = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]$.

*Nota*:

Per poder mostrar els resultats de pèrdua d'informació estaria bé poder
visualitzar les mesures en un gràfic. Per això us proposem utilitzar la
llibreria `seaborn` (que podem importar com` sns`). Per exemple, suposem
que mesurem la pèrdua d'informació a l'aplicar soroll additiu amb diferents
valors de `p`. En aquest cas, tindrem una llista de diccionaris de python
amb la pèrdua d'informació per a cada cas. Per exemple, de forma
simplificada podem tenir la llista següent per als valors de `p` 1 i 2:

```
inf_loss_data = [
  {
    'p': 1,
    'data-mae': 0.1,
    'data-mre': 0.01,
    'corr-mae': 0.2,
    'corr-mre': 0.02
  },
  {
    'p': 2,
    'data-mae': 0.11,
    'data-mre': 0.011,
    'corr-mae': 0.22,
    'corr-mre': 0.022
  }
]
```
que podríem mostrar amb la següent funció:

In [8]:
def plot_information_loss(inf_loss_data: List[Dict[str,float]]):
    df = pd.DataFrame(inf_loss_data)
    melted = df.melt('p', var_name='cols', value_name='vals')
    g = sns.relplot(data=melted, x='p', y='vals', kind='line', col='cols',
                    col_wrap=2, facet_kws=dict(sharey=False))
    return g

In [9]:
# --- IMPLEMENT YOUR CODE HERE ---

...

# --------------------------------

Ellipsis

## **Exercici 5** [10%]

Per solucionar el problema que tenim en el càlcul del MRE, utilitzarem
una altra mesura d'error relatiu. Concretament utilitzarem un percentatge sobre
l'error absolut. Generalment rep el nom de Symmetric Mean Absolute
Percentage Error (SMAPE) i es pot calcular de diverses maneres. En el nostre
cas utilitzarem la següent.

Donades dues matrius $X$ i $X'$ com les que hem anat treballant en aquests
exercicis:
- *MRE-2*: $100 \cdot \frac{\sum_{j=1}^p \sum_{i=1}^n \mid x_{ij} - x'_{ij} \mid}{\sum_{j=1}^p \sum_{i=1}^n (x_{ij} + x'_{ij})}$

Implementeu una nova funció `information_loss_2` de manera anàloga a
l'anterior però que utilitzi aquesta nova mesura de MRE-2 tant per al càlcul de
`data-mre` com el de `corr-mre`.

Amb aquesta nova funció torneu a calcular la pèrdua d'informació obtinguda
a l'aplicar soroll additiu i multiplicatiu anterior:

- Mostra els valors de pèrdua d'informació en cada cas per `df_noise_a`
i `df_noise_m`.

- Mostra de forma gràfica la pèrdua d'informació a l'aplicar soroll additiu
a `df_company` amb valors de $p$ entre $0.1$ i $1.0$ amb un decimal. És a dir, per
$p = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]$.

In [10]:
def information_loss_2(df_original: pd.DataFrame, df_protected: pd.DataFrame) -> Dict[str, float]:
    # --- IMPLEMENT YOUR CODE HERE ---
    
    ...
    
    # --------------------------------

In [11]:
# --- IMPLEMENT YOUR CODE HERE ---

...

# --------------------------------

Ellipsis

## **3. K-anonimitat**

En aquest exercici, treballarem amb un conjunt de dades (`diabetes.csv`) que conté dos variables que podríem considerar quasi-identificadors: edat (`AGE`) i pes (`WEIGHT`). Per tal d'evitar possibles riscos de privacitat mitjançant el creuament amb altres conjunts de dades, aplicarem k-anonimitat a aquest dataset d'entrada utilitzant **microagregació**:

 * Primer, haureu de generar un nou dataset on cada parella edat-pes es repeteixi en, com a mínim, 4 elements del conjunt de dades (k=4)
 
 * En segon lloc, generarem un nou dataset on cada parella edat-pes es repeteixi en, com a mínim, 2 elements (k=2)
 
 * En tercer lloc, calcularem la pèrdua d'informació que hem introduït en aquests dos nous datasets mitjançant els errors absoluts i els errors relatius vistos anteriorment.

En un entorn real, els algorismes habituals que s'utilitzen per aplicar k-anonimitat a un dataset són MDAV o Mondrian. En aquesta pràctica, però, utilitzarem un mètode més senzill perquè pugueu veure un exemple fàcil de com aplicar aquest tipus de protecció.

In [3]:
df_diabetes = pd.read_csv("data/diabetes.csv")
df_diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444 entries, 0 to 443
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      444 non-null    int64  
 1   AGE     444 non-null    int64  
 2   WEIGHT  444 non-null    float64
 3   BMI     444 non-null    float64
 4   BP      444 non-null    float64
 5   S1      444 non-null    int64  
 6   S2      444 non-null    float64
 7   S3      444 non-null    float64
 8   S4      444 non-null    float64
 9   S5      444 non-null    float64
 10  S6      444 non-null    int64  
 11  Y       444 non-null    int64  
dtypes: float64(7), int64(5)
memory usage: 41.8 KB


### Exercici 6 [25%]

En aquest exercici, generarem un nou dataset amb k=4 aplicant microagregació a les variables edat i pes. Per a fer-ho, agregarem primer la variable pes en subconjunts de 20 elements, i després aplicarem microagregació a cada subconjunt resultant a partir de la variable edat, generant nous subconjunts de 4 elements.

A continuació, detallem els passos que podeu seguir.

1. Ordeneu el dataset per la variable 'WEIGHT'
1. Calculeu la mitjana del pes per als primers 20 elements del conjunt.
1. Assigneu el valor de la mitjana obtinguda a tots els 20 elements d'aquest subconjunt.
1. Apliqueu el mateix procediment a la resta dels subconjunts de 20 elements del dataset.

Amb aquestes operacions, haureu obtingut un nou dataset amb la microagregació de la variable pes en conjunts de 20 elements (k=20). A continuació, seguiu els següents passos per fer el mateix sobre la variable edat.

1. Ordeneu ara el dataset per les variables 'WEIGHT' i 'AGE'
1. Calculeu la mitjana de l'edat per als primers 4 elements del conjunt.
1. Assigneu el valor de la mitjana obtingut a tots els 4 elements d'aquest subconjunt.
1. Apliqueu el mateix procediment a la resta dels subconjunts de 4 elements del dataset.

In [ ]:
# --- IMPLEMENT YOUR CODE HERE ---

...

# --------------------------------

### Exercici 7 [5%]

Repetiu el procediment anterior per tal d'obtenir un nou dataset, a partir de l'original (`df_diabetes`), amb k=2.

In [ ]:
# --- IMPLEMENT YOUR CODE HERE ---

...

# --------------------------------

### Exercici 8 [10%]

En aquest exercici, mesurarem la mitjana de l'error absolut MAE (mean absolute error) i la mitjana de l'error relatiu MRE (mean relative error) de cadascun dels 2 nous datasets obtinguts en els exercicis anteriors. Utilitzarem la notació següent:

 * $X$: conjunt de dades original de pacients amb diabetis.
 * $X_4$: conjunt de dades 4-anònim obtingut en primer lloc (exercici 6).
 * $X_2$: conjunt de dades 2-anònim obtingut en segon lloc (exercici 7).

Per a cadascun d'aquests 3 datasets, agafeu només el subconjunt de les variables `AGE` i `WEIGHT` i calculeu:

- `'data_mae_k4'`: valor del MAE sobre $X$ i $X_4$
- `'data_mre_k4'`: valor del MRE sobre $X$ i $X_4$
- `'data_mae_k2'`: valor del MAE sobre $X$ i $X_2$
- `'data_mre_k2'`: valor del MRE sobre $X$ i $X_2$

In [ ]:
# --- IMPLEMENT YOUR CODE HERE ---

...

# --------------------------------

### Exercici 9 [5%]

Feu una reflexió sobre el nivell de privacitat obtingut amb $X_4$ i $X_2$, contrastant-ho amb la pèrdua d'informació introduïda.